In [10]:
import pickle
import hddCRP.behaviorDataHandlers
import matplotlib.pyplot as plt
import numpy as np
import os

In [18]:

context_length = 2;
num_runs = 1;
runs = range(num_runs);
num_warmup_samples = 5000
num_samples        = 5000
overwrite_existing_results = True

simple_model = False
single_concentration = False
sequential_distances_only = True # if false, model setup more as a "smoother". If true, can simulate from the model

if(sequential_distances_only):
    results_directory = "Results/sequentialModel/"
else:
    results_directory = "Results/"
if(simple_model):
    results_directory += "simple/"
elif(single_concentration):
    results_directory += "singleAlpha/"

if(not os.path.exists(results_directory)):
    os.makedirs(results_directory)

data_filename = 'data/Data_turns_all_by_session.pkl';
with open(data_filename, 'rb') as data_file:
    data = pickle.load(data_file)

subjects = list(data["data"].keys())
subjects = ["A1"]
subjects.sort()
print("subjects = " + str(subjects))



subjects = ['A1']


In [19]:

for run_idx in runs:
    for subject_idx, subject in enumerate(subjects):
        filename = "{results_directory}/Subject_{subject_id}_context_{context_length}_run_{run_idx}.pkl".format(results_directory=results_directory, subject_id=subject, run_idx=run_idx, context_length=context_length)
        if(not os.path.isfile(filename) or overwrite_existing_results):
            # for each run, should do some randomization of initial parameters (with known seeds so we can repeat everything)

            seed = subject_idx * 1000 + run_idx;
            rng = np.random.Generator(np.random.MT19937(seed))

            sequences = data["data"][subject]["data"]; # turns in each session
            session_types = data["data"][subject]["task"] # which maze

            ii = session_types.index("C")
            sequences = [sequences[ii]]
            session_types = [session_types[ii]]


            model = hddCRP.behaviorDataHandlers.create_hddCRP(sequences, session_types, rng = rng, 
                        sequential_distances_only=sequential_distances_only, depth=(context_length+1), include_timescales=(not simple_model))

            tau_names = [str(xx) for xx in model.weight_param_labels]
            alphas_names = ["alpha_concentration_no_context", "alpha_concentration_one_back_context", "alpha_concentration_two_back_context"]
            alphas_names = alphas_names[:(context_length+1)]
            model, samples, step_size_settings = hddCRP.behaviorDataHandlers.sample_model_for_maze_data(model, num_samples=num_samples, num_warmup_samples=num_warmup_samples, 
                print_every=100, single_concentration_parameter=(single_concentration or simple_model))
            
            MCMC_info = {"step_size_settings" : step_size_settings.to_dict(),
                        "num_warmup_samples" : num_warmup_samples,
                        "num_samples" : num_samples,
                        "seed" : seed, 
                        "sequential_distances_only" : sequential_distances_only}
            samples["tau_parameter_names"] = tau_names
            samples["alphas_names"] = alphas_names
            samples["stats"] = {"contexts" : model._groupings, "observations" : model._Y}
            
            # save results to filename
            with open(filename, "wb") as results_file:
                results_data = {"MCMC_info" : MCMC_info,
                                "samples" : samples}
                pickle.dump(results_data, results_file)

Sample 0 / 10000
Sample 100 / 10000
Sample 200 / 10000
Sample 300 / 10000
Sample 400 / 10000
Sample 500 / 10000
Sample 600 / 10000
Sample 700 / 10000
Sample 800 / 10000
Sample 900 / 10000
Sample 1000 / 10000
Sample 1100 / 10000
Sample 1200 / 10000
Sample 1300 / 10000
Sample 1400 / 10000
Sample 1500 / 10000
Sample 1600 / 10000
Sample 1700 / 10000
Sample 1800 / 10000
Sample 1900 / 10000
Sample 2000 / 10000
Sample 2100 / 10000
Sample 2200 / 10000
Sample 2300 / 10000
Sample 2400 / 10000
Sample 2500 / 10000
Sample 2600 / 10000
Sample 2700 / 10000
Sample 2800 / 10000
Sample 2900 / 10000
Sample 3000 / 10000
Sample 3100 / 10000
Sample 3200 / 10000
Sample 3300 / 10000
Sample 3400 / 10000
Sample 3500 / 10000
Sample 3600 / 10000
Sample 3700 / 10000
Sample 3800 / 10000
Sample 3900 / 10000
Sample 4000 / 10000
Sample 4100 / 10000
Sample 4200 / 10000
Sample 4300 / 10000
Sample 4400 / 10000
Sample 4500 / 10000
Sample 4600 / 10000
Sample 4700 / 10000
Sample 4800 / 10000
Sample 4900 / 10000
Sample 5000 

In [ ]:
# example for loading results and computing estimates with error bars for a single subject
subject = 'A1'


taus = np.zeros((10000,2,4))
alphas = np.zeros((10000,3,4))

for run_idx in range(4):
    filename = "{results_directory}/Subject_{subject_id}_run_{run_idx}.pkl".format(results_directory=results_directory, subject_id=subject, run_idx=run_idx)
    with open(filename, "rb") as file:
        results = pickle.load(file)

        ss = range(results["MCMC_info"]["num_warmup_samples"], results["MCMC_info"]["num_warmup_samples"]+results["MCMC_info"]["num_samples"])
        taus[:,:,run_idx] = np.exp(results["samples"]["log_taus"][ss,:]).squeeze()
        alphas[:,:,run_idx] = results["samples"]["alphas"][ss,:]

        # ci_range = [2.5, 97.5]

        # mean_taus = np.mean(taus, axis=0)
        # mean_alphas = np.mean(alphas, axis=0)
        # std_taus = np.std(taus, axis=0)
        # std_alphas = np.std(alphas, axis=0)
        # ci95_taus = np.percentile(taus, ci_range, axis=0)
        # ci95_alphas = np.percentile(alphas, ci_range, axis=0)

        # accepted = results["samples"]["accepted"][ss]
        # print("Fraction of accepted MH samples: " + str(np.mean(accepted)))

    # plt.subplot(1,2,1)
    # plt.plot(taus)

    # plt.subplot(1,2,2)
    # plt.plot(alphas)
    # plt.show()


In [ ]:
from itertools import product
print(model._groupings[:5,:])
# print(model._Y)

prefixes = [str(xx) + '-' for xx in np.unique(model._Y)]
combinations = list(product(prefixes, prefixes))

level_0_grps = ['' for xx in combinations]
level_1_grps = [''.join(xx[-1:]) for xx in combinations]
level_2_grps = [''.join(xx[-2:]) for xx in combinations]

print(level_0_grps)
print(level_1_grps)
print(level_2_grps)


g_ns_level_0 = [int(model._groupings_compact[model._groupings[:,0] == xx,0][0]) if np.any(np.isin(xx,model._groupings[:,0])) else model.num_groups[0] for xx in level_0_grps]
g_ns_level_1 = [int(model._groupings_compact[model._groupings[:,1] == xx,1][0]) if np.any(np.isin(xx,model._groupings[:,1])) else model.num_groups[1] for xx in level_1_grps]
g_ns_level_2 = [int(model._groupings_compact[model._groupings[:,2] == xx,2][0]) if np.any(np.isin(xx,model._groupings[:,2])) else model.num_groups[2] for xx in level_2_grps]

gnums = [g_ns_level_0, g_ns_level_1, g_ns_level_2]
print(g_ns_level_0)
print(g_ns_level_1)
print(g_ns_level_2)

In [ ]:
from hddCRP.modelFitting import complete_exponential_distance_function_for_maze_task, hddCRPModel


obs_idx = model.N-1
n_obs = np.size(obs_idx)

D_new = np.array(model._D[obs_idx,:,:].reshape((n_obs,model.N,1)))
inds_new = model._weight_function_setup["inds"][obs_idx,:].reshape((n_obs,model.N))
D_new[inds_new == 0] += 1

weights = complete_exponential_distance_function_for_maze_task(D_new, model.weight_params, inds_new, model._weight_function_setup["timescale_inds"],model._weight_function_setup["constant_scale_inds"])
#D_new.shape
# inds_new
print(weights)

In [17]:


context_length

1